# Grundlegende Algorithmen und Verfahren der KI - Programmentwurf 12
Q1 2024 TIK

Gruppe:
- Simon Seifert
- Jonas Preusch
- Daniel Zängler
- Johannes Hoppe

## Aufgabe: Partitionsproblem
Gegeben ist eine Menge von $N$ Objekten mit den Größen $a_1,a_2,...,a_N \in \mathbb{N}$. Gesucht ist eine Teilmenge $P \subseteq \{1,2,...,N\}$ der Objekte, für welche $\sum{_{i \in P}}\ a_i = \sum{_{i \notin P}}\ a_i$ gilt.
Mit einem evolutionären Algorithmus soll eine solche Partition $P$ gefunden werden.

Mit einem evolutionären Algorithmus soll eine derartige Partition P gefunden werden. Für die
Erstellung einer Implementierung sind folgende Anforderungen zu beachten.
- Ausgangspunkt ist eine Folge von Größen $a_1, a_2,..., a_N \in \mathbb{N}$.
- Als genetische Repräsentation eines Lösungsvorschlags dient eine binäre Folge der Länge $N$. Dabei gilt $i \in P$ genau dann, wenn die $i$-te Stelle der Folge eine 1 ist.
- Implementieren Sie die Transformationen Selektion, Mutation und Rekombination.
- Wählen Sie $N \geq 10$ und starten Sie mit einer initialen Population von mindestens 50
Individuen.

---

## Programmentwurf:
Für die Lösung dieses Problems haben wir uns für die Pogrammiersprache Python entschieden.
Unser Code ist dabei in die folgenden Dateien aufgeteilt:
- [`main.py`](main.py)
- [`back_end.py`](back_end.py)
- [`algorithm.py`](algorithm.py)
- [`util.py`](util.py)
- [`config_storage.yaml`](config_storage.yaml)
- [`set_storage.yaml`](set_storage.yaml)

Sowie diesem Python Notebook, im Jupyter .ipynb Format, welches der Dokumentation des Entwurfs dient.

### Datei erläuterung:

`main.py` dient zum ausführen des Programms.

`back_end.py` enthält die grundlegenden Strukturen, welche wir für einen evolutionären Algorithmus benötigen (Selektion, Mutation, Rekombination), sowie Funktionen zur bestimmung der 'Fitness' der Lösungen.

`algorithm.py` enthält den eigentlichen Teil des Algorithmus und ruft dafür die Funktionen aus `back_end.py` auf.

`util.py` enthält Funktionen, die das Speichern/Laden von verschieden Konfigurationen ermöglichen, sowie eine Funktion um die Ergebnisse schlussendlich in einem 'Pie-Chart' darzustellen.

`config_storage.yaml` und `set_storage.yaml` enthalten die Initiale Konfiguration und Datensatz, für das Programm.

---

## Code:

Die folgende Code Erläuterung findet bestmöglich in der selben Reihenfolge statt, die auch das Programm bei Ausführung folgt. 

#### `main()`
Beim Ausführen des Programms werden die Funktionen aus `algorithm.py` und `util.py` importiert und die `main()` Funktion aufgerufen, vorrausgesetzt die Datei ist `"__main__"`, also wenn sie ist die Datei, die von Python gestartet wird.

Zunächst wird mit `util.load_set()` der Datensatz, über welchen der Algorithmus laufen soll, sowie mit `util.load_config()` die initialen Werte aus `config_storage.yaml` geladen.
Diese sind:

- `amount_generations` : Anzahl an Generationen, für die der Algorithmus laufen soll
- `fitness_selection` : Anzahl an Lösungen einer Generation, die verändert und für die nächste verwendet werden sollen.
- `generation_size` : Anzahl der Individuen pro Generation
- `good_enough_number` : Fitness-Wert, ab dem eine (ausreichende / 'good enough') Lösung gefunden wurde, sodass vor erreichen der festgelegten maximalen Laufzeit abgebrochen werden kann.
- `mutation_factor` : Anzahl der Mutationen, pro Durchführung der Mutations-Funktion
- `stagnation_threshold` : TODO 

In [ ]:
import algorithm
import util


def main():
    print(util.get_all_configs())

    set_1 = util.load_set(1)
    run_config2 = util.load_config(1)
    results = algorithm.run_algorithm_multiple_times(set_1, *run_config2.values())
    util.plot_generations_needed(results)


if __name__ == "__main__":
    main()

`run_algorithm_multiple_times()` wird mit den zuvor geladenen Werten aufgerufen.

#### `run_algorithm_multiple_times()`
`run_algorithm_multiple_times()` ruft den eigetntlichen Algorithmus mit `run_evolutionary_algorithm()` mehrfach auf, bis zur Anzahl definitert in `num_runs`. Dabei werden die Restlichen Werte einfach an die Funktion weitergegeben.
Die Ergebnisse werden in einer Liste gespeichert und zurück gegeben.  

In [ ]:
def run_algorithm_multiple_times(initial_set, generation_size, amount_generations, fitness_selection, mutation_factor,
                                 stagnation_threshold, num_runs, good_enough_number):
    results = []

    for _ in range(num_runs):
        result = run_evolutionary_algorithm(initial_set, generation_size, amount_generations, fitness_selection,
                                            mutation_factor, stagnation_threshold,
                                            good_enough_number)
        results.append(result)

    return results

#### `run_evolutionary_algorithm()`
`run_evolutionary_algorithm()` ist die hautsächliche Funktion des Programms.

1. Variablen werden definiert und zunächst gesetzt:
    - Die `population` (durch [`initialization()`](#initialization)) geladen.
    - Die Lösung mit der momentan besten Fitness wird als `best_solution` gesetzt.
    - `stagnation_tracker` wird definiert.
2. Die Funktionen werden `generation_size` oft wiederholt:
    - Die Population wird nach Fitness sortiert.
    - Falls bereits eine Lösung mit ausreichender Fitness gefunden wurde, wird diese zurück gegeben und die Funktion beendet.
    - **TODO**: `stagnation_threshold()`
    - Die besten Individuen werden für die Mutation und Rekombination ausgewählt (Selektion)
    - Diese Individuen werden mutiert ([`mutate()`](#mutate))
    - Diese Individuen werden rekombiniert ([`random_crossover()`](#mutate))
    - Danach erneut sortiert
    - Falls die beste dadurch entstandene Lösung besser ist, als die momentan Beste in `best_solution`, wird `best_solution` mit der neuen überschrieben.

In [ ]:
import back_end


def run_evolutionary_algorithm(initial_set, amount_solutions, generations, fitness_selection=10, mutation_factor=4,
                               stagnation_threshold=3, good_enough_number=2):
    # Initialize the population
    population = back_end.initialization(initial_set, amount_solutions)
    best_solution = back_end.sort_by_fitness(population)[0]  # Get the best solution from the sorted population
    stagnation_tracker = 0

    # Evolve the population for the specified number of generations
    for gen in range(generations):
        population = back_end.sort_by_fitness(population)
        # if we achieved a good enough solution abort
        if best_solution.fitness <= good_enough_number:  # Check the fitness of the best solution
            print(f"found good enough solution: {best_solution.fitness}")

            return {"solution": best_solution, "generations_needed": gen + 1}
        if stagnation_tracker >= stagnation_threshold:
            print("stagnation threshold reached")
            population = back_end.break_stagnation(population)
            stagnation_tracker = 0
        stagnation_tracker += 1
        # Select the best individuals fraction
        population = back_end.best_fitness_selection(population, fitness_selection)
        print(f"best Score: {best_solution.fitness}")
        # Apply mutation to the selected individuals
        population = back_end.mutate(population, mutation_factor=mutation_factor)

        # Perform crossover to generate the next generation
        population = back_end.random_crossover(population, amount_solutions)
        population = back_end.sort_by_fitness(population)

        # Update the best solution if necessary
        if population[0].fitness < best_solution.fitness:  # Check the fitness of the best solution
            best_solution = population[0]

    return {"solution": best_solution, "generations_needed": generations}

#### `initialization()`

-> Bringt Werte aus dem initialen Datensatz in Form der Klasse [`Solution`](#solution)

In [ ]:
def initialization(initial_set, amount_solutions):
    return [Individual(initial_set) for _ in range(amount_solutions)]

#### `Solution`
Die Klasse besteht aus den Teilen:
- `initial_set` : Initialer Datensatzt
- `solution` : Zufällig generierte binäre Abfolge, durch [`generate_solutions()`](#generate_solutions)
- `total_sum` : Summe der Werte aus `initial_set` (zum Zweck der Berechnung der Fitness)
- `partial_sum` : Teilwert **TODO** (zum Zweck der Berechnung der Fitness)
- `fitness` : Bewertung der Lösung im Vergleich zum ideal Ergebnis, durch [`calculate_fitness`](#calculate_fitness)

Auch enthält sie einen Teil um als 'String' ausgegeben werden zu können.

In [ ]:
import random

class Individual:
    def __str__(self):
        return (
            f"Solution: initial_set={self.initial_set}, solution={self.solution}, total_sum={self.total_sum},"
            f"partial_sum={self.partial_sum}, fitness={self.fitness}"
        )

    # first implementation of the fitness function
    def calculate_fitness(self):
        if (self.partial_sum - self.total_sum / 2) == 0:
            return 100
        fitness = (abs(self.partial_sum - (self.total_sum / 2)))
        return fitness

    def calculate_partial_sum(self):
        partial_sum = 0
        for i in range(len(self.initial_set)):
            if self.solution[i] == 1:
                partial_sum += self.initial_set[i]
        return partial_sum

    def generate_solution(self):
        return [random.choice([0, 1]) for _ in self.initial_set]

    # initial Set is always bigger 10 so no division by 0 possible in fitness
    def __init__(self, initial_set):
        self.initial_set = initial_set
        self.solution = self.generate_solution()
        self.total_sum = sum(initial_set)
        self.partial_sum = self.calculate_partial_sum()
        self.fitness = self.calculate_fitness()

#### `calculate_fitness()`
-> Berechnung der Fitness


In [ ]:
def calculate_fitness(self):
        if (self.partial_sum - self.total_sum / 2) == 0:
            return 100
        fitness = (abs(self.partial_sum - (self.total_sum / 2)))
        return fitness

#### `calculate_parital_sum()`


#### `generate_solution()`

#### `best_fitness_selection()`
Diese Funktion ist für den Selektion in unserem Algorithmus zuständig.
`best_fitness_selection()` sortiert die Lösungen anhand ihrer 'Fitness' und gibt die Top-10 dieser zurück.
(Überschreibt die `population`-Variable)

In [ ]:
def best_fitness_selection(solutions, output_amount):
    return sorted(solutions,key=lambda x: x.fitness, reverse=True)[:output_amount]

#### `mutate()`
Diese Funktion ist für Mutation in unserem Algorithmus zuständig.
`mutate()` ändert eine zufällige Stelle der binären Abfolge (0->1 oder 1->0)

**TODO** Anstelle von `random.choice([0, 1])` könnte der Betrag von `solution[i] - 1` genommen werden, damit eine Änderung von 1->1 oder 0->0 nicht mehr möglich ist.
(Überschreibt die `population`-Variable)

In [ ]:
import random
def mutate(solutions, mutation_factor=1):
    # TODO: check if mutation from 1 to 1 should be possilbe, or to just flip values
    # chose a mutation_factor between 0 and length of the solution. we will then choose random positions of solution to
    # redraw the values,the amount based on the mutation_factor
    # we use range() plus len() to choose which positions of the solutions to change
    for solution in solutions:
        mutations = random.sample(range(len(solution.solution)), mutation_factor)
        for i in mutations:
            solution.solution[i] = random.choice([0, 1])
    return solutions

#### `random_crossover()`
Diese Funktion ist für die Rekombination in unserem Algorithmus zuständig.
`random_crossover()` wählt aus der `population` zufällig zwei 'Eltern' aus, von denen zufällige 'Gen-Paare' miteinader getauscht werden um eine neue Lösung zu generieren.
(Überschreibt die `population`-Variable)

In [ ]:
def random_crossover(parent_solutions, num_children):
    """
    Perform random crossover between a list of parent solutions.

    This method randomly selects each gene from either parent with equal probability
    to produce a specified number of child solutions.

    Args:
    - parent_solutions (list): A list of parent solutions, where each solution is represented as a list.
    - num_children (int): The number of child solutions to generate.

    Returns:
    - list: A list of child solutions resulting from the crossover operation.
    """


    children = []
    for _ in range(num_children):
        # Randomly select parents
        parent1 = random.choice(parent_solutions)
        parent2 = random.choice(parent_solutions)

        # Perform random crossover
        child=Solution(initial_set=parent1.initial_set)
        child.solution = [random.choice(gene_pair) for gene_pair in zip(parent1.solution, parent2.solution)]

        children.append(child)

    return children

#### `run_evolutionary_algorithm()` -> Ergebnis & return
([Hauptabschnitt](#run_evolutionary_algorithm))

Nach Selektion, Mutation, und Rekombination der ganzen Population, bis zur Anzahl der definierten Generationen, wird die endgültige Population noch ein mal an hand der 'Fitness' sortiert und an [`find_best_solution`](#find_best_solution) zurück gegeben:
```python
    # Sort the final population by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)

    return population
```

#### `find_best_solution()` -> Ergebnis & return

([Hauptabschnitt](#find_best_solution))

Mit der endgültigen Population, von [`run_evolutionary_algorithm()`](#run_evolutionary_algorithm---ergebnis--return) wird der Eintrag mit der besten 'Fitness' ausgewählt und an [`main()`](#main) zurück gegeben.
```python
    # Select the best solution from the final population
    best_solution = back_end.best_fitness_selection(result_population, 1)[0]

    return best_solution
```

#### `main()` -> Ergebnis

([Hauptabschnitt](#main))

`main()` gibt zuletzt das beste gefundene Ergebniss als Pie-Chart aus, und das Programm wird beendet.
```python
    util.plot_generations_needed(results)
```